In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'H2'#'LiH'
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]#[('Li', (0., 0., 0.)), ('H', (0., 0., 1.45))]
num_shots = 10000
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='JW')
### HAMILTONIAN end
QubitHamiltonian

(-0.09706626861762624+0j) [] +
(-0.04530261550868928+0j) [X0 X1 Y2 Y3] +
(0.04530261550868928+0j) [X0 Y1 Y2 X3] +
(0.04530261550868928+0j) [Y0 X1 X2 Y3] +
(-0.04530261550868928+0j) [Y0 Y1 X2 X3] +
(0.17141282639402405+0j) [Z0] +
(0.1686889816869329+0j) [Z0 Z1] +
(0.12062523481381837+0j) [Z0 Z2] +
(0.16592785032250768+0j) [Z0 Z3] +
(0.171412826394024+0j) [Z1] +
(0.16592785032250768+0j) [Z1 Z2] +
(0.12062523481381837+0j) [Z1 Z3] +
(-0.2234315367466397+0j) [Z2] +
(0.174412876106516+0j) [Z2 Z3] +
(-0.2234315367466397+0j) [Z3]

In [2]:
################## get sets for UP

Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 15/15 [00:00<00:00, 1646.59it/s]


{0: [(0.1686889816869329+0j) [Z0 Z1]],
 1: [(0.16592785032250768+0j) [Z1 Z2]],
 2: [(0.16592785032250768+0j) [Z0 Z3]],
 3: [(0.12062523481381837+0j) [Z0 Z2]],
 4: [(0.174412876106516+0j) [Z2 Z3]],
 5: [(0.12062523481381837+0j) [Z1 Z3]],
 6: [(-0.09706626861762624+0j) []],
 7: [(-0.04530261550868928+0j) [X0 X1 Y2 Y3], (-0.2234315367466397+0j) [Z3]],
 8: [(0.04530261550868928+0j) [Y0 X1 X2 Y3], (0.17141282639402405+0j) [Z0]],
 9: [(0.04530261550868928+0j) [X0 Y1 Y2 X3], (0.171412826394024+0j) [Z1]],
 10: [(-0.2234315367466397+0j) [Z2], (-0.04530261550868928+0j) [Y0 Y1 X2 X3]]}

In [3]:
######### Ansatz circuit

from quchem.Simulating_Quantum_Circuit import *
from quchem.Ansatz_Generator_Functions import *
from openfermion.ops import QubitOperator

def H2_ansatz(theta):
    HF_circ = [cirq.X.on(cirq.LineQubit(0)), cirq.X.on(cirq.LineQubit(1))]
    
    full_exp_circ_obj = full_exponentiated_PauliWord_circuit(QubitOperator('Y0 X1 X2 X3', -1j), theta)
    UCCSD_circ = cirq.Circuit(cirq.decompose_once((full_exp_circ_obj(*cirq.LineQubit.range(full_exp_circ_obj.num_qubits())))))
    full_circuit = cirq.Circuit([*HF_circ, *UCCSD_circ.all_operations()])
    
    return full_circuit
    
H2_ansatz(np.pi)  

0: ───X───Rx(0.5π)───@──────────────────────────────@───Rx(-0.5π)───
                     │                              │
1: ───X───H──────────X───@──────────────────────@───X───H───────────
                         │                      │
2: ───H──────────────────X───@──────────────@───X───H───────────────
                             │              │
3: ───H──────────────────────X───Rz(2.0π)───X───H───────────────────

In [4]:
from quchem.LCU_method import *
from quchem.Unitary_partitioning import * 
from quchem.quantum_circuit_functions import *

In [17]:
def Get_pauli_matrix(PauliOp, N_system_qubits):
    
    pauliDict=   {'X':np.array([[0,1],[1,0]]),
                          'Y':np.array([[0,-1j],[1j,0]]),
                          'Z':np.array([[1,0],[0,-1]]),
                          'I': np.eye(2)}
    
    list_Q_nos, list_P_strs = list(zip(*[Paulistrs for Paulistrs, const in PauliOp.terms.items()][0]))

    list_of_ops = []
    for i in range(N_system_qubits):
        if i in list_Q_nos:
            index = list_Q_nos.index(i)
            list_of_ops.append(pauliDict[list_P_strs[index]])
        else:
            list_of_ops.append(pauliDict['I'])

    matrix = reduce(kron, list_of_ops)

    return matrix

In [97]:
def Calc_Pauli_expect_of_set_LCU(theta, Pn_index, anti_commuting_set):
    
    R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_commuting_set, Pn_index)
    R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1_norm = absorb_complex_phases(R_uncorrected)
    
    ansatz_circuit = H2_ansatz(theta)
        
    LCU_Q_circuit = Full_Ansatz_and_Quantum_R_circuit(Pn,
                                   R_corrected_Op_list,
                                   R_corr_list,
                                   ancilla_amplitudes,
                                   Hamilt.molecule.n_qubits,
                                   ansatz_circuit)
    
    
    input_state = [np.array([[1], [0]]) for _ in range(len(LCU_Q_circuit.all_qubits()))]
    input_ket = reduce(kron, input_state)
    circuit_matrix = LCU_Q_circuit.unitary()

    ansatz_state_ket = circuit_matrix.dot(input_ket.todense())

    full_density_matrix = np.outer(ansatz_state_ket, ansatz_state_ket)


    ## First project state onto all zero ancilla state using POVM
    n_qubits = len(LCU_Q_circuit.all_qubits())
    n_ancilla = int(np.ceil(np.log2(len(ancilla_amplitudes))))
    N_system_qubits = n_qubits - n_ancilla

    I_system_operator = np.eye((2**N_system_qubits))

    ancilla_0_state_list = [np.array([[1], [0]]) for _ in range(n_ancilla)]
    ancilla_0_state = reduce(np.kron, ancilla_0_state_list)
    ancilla_0_projector = np.outer(ancilla_0_state, ancilla_0_state)

    POVM_0_ancilla = np.kron(I_system_operator, ancilla_0_projector)
    Kraus_Op_0 = POVM_0_ancilla.copy()

    term = Kraus_Op_0.dot(full_density_matrix.dot(Kraus_Op_0.transpose().conj()))
    projected_density_matrix = term/np.trace(term) # projected into correct space using POVM ancilla measurement!

    ## Next get partial density matrix over system qubits # aka partial trace!
    # https://scicomp.stackexchange.com/questions/27496/calculating-partial-trace-of-array-in-numpy

    # reshape to do the partial trace easily using np.einsum
    reshaped_dm = projected_density_matrix.reshape([2 ** N_system_qubits, 2 ** n_ancilla,
                                                    2 ** N_system_qubits, 2 ** n_ancilla])
    reduced_dm = np.einsum('jiki->jk', reshaped_dm)
    
    
    H_sub_term_matrix = Get_pauli_matrix(Pn, N_system_qubits)
    
    energy = np.trace(reduced_dm.dot(H_sub_term_matrix.todense()))
    
    return (energy * gamma_l).real


Pn_index=0
set_index=10
theta=np.pi
Calc_Pauli_expect_of_set_LCU(theta, Pn_index, anti_commuting_sets[set_index])

-0.22343153674663976

In [38]:
def Calc_Pauli_expect_of_set_CONJ(theta, PS_index, anti_commuting_set):
    
    normalised_set = Get_beta_j_cofactors(anti_commuting_set)
    X_sk_dict = Get_X_sk_operators(normalised_set, S=PS_index)
    
    ansatz_circuit = H2_ansatz(theta)
    

    CONJ_Q_circuit = Generate_Full_Q_Circuit_unitary_part_NO_M_gates(ansatz_circuit,
                                                    X_sk_dict)
    
    input_state = [np.array([[1], [0]]) for _ in range(len(CONJ_Q_circuit.all_qubits()))]
    input_ket = reduce(kron, input_state)
    circuit_matrix = CONJ_Q_circuit.unitary()

    ansatz_state_ket = circuit_matrix.dot(input_ket.todense())
    
    ansatz_state_bra = ansatz_state_ket.transpose().conj()
    H_sub_term_matrix = Get_pauli_matrix(X_sk_dict['PauliWord_S'], len(CONJ_Q_circuit.all_qubits()))
    
    energy = ansatz_state_bra.dot(H_sub_term_matrix.dot(ansatz_state_ket))
    
    
    return (energy.item(0) * X_sk_dict['gamma_l'])


PS_index=0
set_index=10
theta=np.pi
Calc_Pauli_expect_of_set_CONJ(theta, PS_index, anti_commuting_sets[set_index])

(-0.22343153674663985+0j)

In [84]:
def Calc_Pauli_expect_of_set_standard(theta, PauliWord):
    
    if list(PauliWord.terms.keys())[0] ==():
        factor = list(PauliWord.terms.values())[0]
        return factor
    else:
        ansatz_circuit = H2_ansatz(theta)

        input_state = [np.array([[1], [0]]) for _ in range(len(ansatz_circuit.all_qubits()))]
        input_ket = reduce(kron, input_state)
        circuit_matrix = ansatz_circuit.unitary()

        ansatz_state_ket = circuit_matrix.dot(input_ket.todense())
        ansatz_state_bra = ansatz_state_ket.transpose().conj()

        H_sub_term_matrix = Get_pauli_matrix(PauliWord, len(ansatz_circuit.all_qubits()))

        exp = ansatz_state_bra.dot(H_sub_term_matrix.dot(ansatz_state_ket))
        factor = list(PauliWord.terms.values())[0]

        energy = (exp.item(0) * factor)

        return energy

Calc_Pauli_expect_of_set_standard(np.pi, anti_commuting_sets[6][0])

(-0.09706626861762624+0j)

In [95]:
Pn_index=0
PS_index=0
theta=np.pi

LCU_all_vals=[]
Conj_all_vals=[]
Standard_all_vals=[]

for anti_commuting_set in anti_commuting_sets.values():
    if len(anti_commuting_set)>1:
        
        LCU_E = Calc_Pi_expect_of_set_LCU(theta, Pn_index, anti_commuting_set)
        Conj_E = Calc_Pi_expect_of_set_LCU(theta, PS_index, anti_commuting_set)
        
        E_standard_list=[]
        for PauliOp in anti_commuting_set:
            E_standard_list.append(Calc_Pauli_expect_of_set_standard(theta, PauliOp))
        
        standard_E=sum(E_standard_list)
        LCU_all_vals.append(LCU_E)
        Conj_all_vals.append(Conj_E)
        Standard_all_vals.append(standard_E)
        
    else:
        
        energy = Calc_Pauli_expect_of_set_standard(theta, anti_commuting_set[0])
        
        LCU_all_vals.append(energy)
        Conj_all_vals.append(energy)
        Standard_all_vals.append(energy)
    

In [96]:
print(sum(Standard_all_vals))
print(sum(LCU_all_vals))
print(sum(Conj_all_vals))

(-1.1167593073781574+0j)
(-1.1167593073781572+0j)
(-1.1167593073781572+0j)
